In [250]:
import pandas as pd
import csv
import datetime
import sklearn.model_selection

In [ ]:
# This will work 95% of the time, but I already had everything formatted, so I kept my stuff. BUT super useful in future.
pd.read_csv('takehome_users.csv', encoding = 'ISO-8859-1')

In [99]:
# The takehome_users.csv file was being a meanie so I did my best to circumvent the issue while still getting it in pd df form
users = open('takehome_users.csv')
users = pd.DataFrame(csv.reader(users))
users.columns = users.iloc[0]
users = users.drop([0,0])
users.object_id = pd.to_numeric(users.object_id)
users.opted_in_to_mailing_list = pd.to_numeric(users.opted_in_to_mailing_list)
users.enabled_for_marketing_drip = pd.to_numeric(users.enabled_for_marketing_drip)
users.invited_by_user_id = pd.to_numeric(users.invited_by_user_id)
users.creation_time = pd.to_datetime(users.creation_time, format = '%Y-%m-%d %H:%M:%S')
users.invited_by_user_id = users.invited_by_user_id.fillna(0)
users = users.drop(columns = ['name','email','last_session_creation_time'])

In [ ]:
engagement = pd.read_csv('takehome_user_engagement.csv')
engagement.time_stamp = pd.to_datetime(engagement.time_stamp, format = '%Y-%m-%d %H:%M:%S')
dates=[]
for i in range(len(engagement)):
    dates.append(engagement.time_stamp[i].date().strftime('%V-%Y'))
engagement['dates'] = dates

In [183]:
count_df = engagement.groupby(['user_id','dates']).size().reset_index().rename(columns={0:'count'})
adopted = []
for i in range(len(count_df)):
    if count_df['count'][i] >= 3:
        adopted.append(1)
    else:
        adopted.append(0)
count_df['adopted'] = adopted
count_df = count_df.sort_values(by = ['user_id','adopted'], ascending = [True, False]).reset_index(drop = True)

In [247]:
proper = []
for i in range(1,12000):
    proper.append(i)
proper = pd.DataFrame(proper)
name = count_df.groupby('user_id')['adopted'].sum()
name = pd.DataFrame(name)
full = proper.merge(name, how = 'outer', left_on = 0, right_on ='user_id').fillna(0)
final = []
for i in range(len(full)):
    if full.adopted[i] != 0:
        final.append(1)
    else:
        final.append(0)
full['final'] = final

In [248]:
final_df = users.merge(full, left_on = 'object_id', right_on = 0)
final_df = final_df.drop(columns = [0,'adopted','invited_by_user_id'])
final_df = pd.get_dummies(final_df, columns = ['creation_source'], dummy_na = True, drop_first = True)

In [255]:
from sklearn.model_selection import train_test_split
Y = final_df['final']
X = final_df.drop(columns = ['final','creation_time'])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 69)

In [254]:
X_train

,object_id,creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,creation_source_nan
8381,8382,2013-10-16 09:55:32,0,0,75,0,0,1,0,0
11413,11414,2013-06-14 12:08:41,0,1,196,0,1,0,0,0
6534,6535,2013-01-02 00:18:48,0,0,0,0,1,0,0,0
5705,5706,2013-03-08 19:24:11,0,0,9,1,0,0,0,0
6489,6490,2013-10-19 15:30:38,0,0,292,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
8631,8632,2013-01-26 12:02:35,0,0,194,0,1,0,0,0
9818,9819,2012-09-15 14:49:50,0,0,105,0,1,0,0,0
10859,10860,2013-07-05 04:41:51,0,0,408,0,1,0,0,0
4041,4042,2013-03-10 10:42:12,1,1,64,1,0,0,0,0


In [258]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
precision = precision_recall_fscore_support(Y_test, y_pred)
print(clf.feature_importances_)
precision

[0.57970755 0.03334432 0.02836589 0.32153667 0.00621023 0.0055776
 0.00940769 0.01585006 0.        ]


(array([0.88265306, 0.14224138]),
 array([0.87428932, 0.15207373]),
 array([0.87845129, 0.14699332]),
 array([3166,  434], dtype=int64))